E7. Transportation simplex algorithm

Hiba Nejjari

In [1]:
file_path = 'Transportation_Problem.csv'

In [2]:
import pandas as pd
def read_and_display_data(file_path):
    #read the CSV file
    df = pd.read_csv(file_path, index_col=0)
    #display the dataframe
    print("Data from the CSV file created:")
    print(df)
read_and_display_data(file_path)

Data from the CSV file created:
        D1  D2  D3  D4  Supply
S1       4   3   2   8    50.0
S2       6   7   5   3    60.0
S3       9   4   5   2    25.0
Demand  20  30  40  45     NaN


In [3]:
import numpy as np
import heapq

def read_transportation_data(file_path):
    df = pd.read_csv(file_path, index_col=0)
    cost_matrix = df.iloc[:-1, :-1].values.astype(float)
    supply = df.iloc[:-1, -1].values.astype(int)
    demand = df.iloc[-1, :-1].values.astype(int)
    return cost_matrix, supply, demand

def northwest_corner(supply, demand):
    i, j = 0, 0
    solution = np.zeros_like(supply[:, None] + demand)
    while i < len(supply) and j < len(demand):
        allocation = min(supply[i], demand[j])
        solution[i, j] = allocation
        supply[i] -= allocation
        demand[j] -= allocation
        if supply[i] == 0:
            i += 1
        if demand[j] == 0:
            j += 1
    return solution

def minimum_cost_method(cost_matrix, supply, demand):
    remaining_indices = np.indices(cost_matrix.shape)
    solution = np.zeros_like(cost_matrix)
    while np.any(supply) and np.any(demand):
        min_index = divmod(cost_matrix.argmin(), cost_matrix.shape[1])
        i, j = min_index
        allocation = min(supply[i], demand[j])
        solution[i, j] = allocation
        supply[i] -= allocation
        demand[j] -= allocation
        cost_matrix[i, j] = np.inf
    return solution

def print_solution(solution, method_name):
    print(f"\n{method_name} Solution:")
    print(pd.DataFrame(solution, columns=[f"D{j+1}" for j in range(solution.shape[1])],
                       index=[f"S{i+1}" for i in range(solution.shape[0])]))

def solve_transportation_problem(file_path):
    cost_matrix, supply, demand = read_transportation_data(file_path)
    methods = {
        "Northwest Corner Rule": northwest_corner(supply.copy(), demand.copy()),
        "Minimum Cost Method": minimum_cost_method(cost_matrix.copy(), supply.copy(), demand.copy())
    }
    for name, solution in methods.items():
        print_solution(solution, name)
solve_transportation_problem(file_path)



Northwest Corner Rule Solution:
    D1  D2  D3  D4
S1  20  30   0   0
S2   0   0  40  20
S3   0   0   0  25

Minimum Cost Method Solution:
      D1    D2    D3    D4
S1   0.0  10.0  40.0   0.0
S2  20.0  20.0   0.0  20.0
S3   0.0   0.0   0.0  25.0


In [4]:
def minimum_row_cost_method(cost_matrix, supply, demand):
    solution = np.zeros_like(cost_matrix)
    for i in range(len(supply)):
        while supply[i] > 0:
            j = np.argmin(cost_matrix[i])
            allocation = min(supply[i], demand[j])
            solution[i, j] = allocation
            supply[i] -= allocation
            demand[j] -= allocation
            cost_matrix[i, j] = np.inf
    return solution

def print_solution(solution, method_name):
    print(f"\n{method_name} Solution:")
    print(pd.DataFrame(solution, columns=[f"D{j+1}" for j in range(solution.shape[1])],
                       index=[f"S{i+1}" for i in range(solution.shape[0])]))

def solve_transportation_problem(file_path):
    cost_matrix, supply, demand = read_transportation_data(file_path)
    print_solution(minimum_row_cost_method(cost_matrix.copy(), supply.copy(), demand.copy()), "Minimum Row Cost Method")
solve_transportation_problem(file_path)



Minimum Row Cost Method Solution:
      D1    D2    D3    D4
S1   0.0  10.0  40.0   0.0
S2  15.0   0.0   0.0  45.0
S3   5.0  20.0   0.0   0.0


In [5]:
def vogel_approximation_method(cost_matrix, supply, demand):
    solution = np.zeros_like(cost_matrix)
    costs = cost_matrix.copy()
    
    while np.any(supply > 0) and np.any(demand > 0):
        row_penalties = []
        col_penalties = []
        
        for row in costs:
            filtered_row = row[row != np.inf]
            if len(filtered_row) > 1:
                sorted_row = np.sort(filtered_row)
                row_penalties.append(sorted_row[1] - sorted_row[0])
            elif len(filtered_row) == 1:
                row_penalties.append(0)  # No penalty if only one element
            else:
                row_penalties.append(-1)  # Indicate depleted row
        
        for j in range(costs.shape[1]):
            filtered_col = costs[:, j][costs[:, j] != np.inf]
            if len(filtered_col) > 1:
                sorted_col = np.sort(filtered_col)
                col_penalties.append(sorted_col[1] - sorted_col[0])
            elif len(filtered_col) == 1:
                col_penalties.append(0)  # No penalty if only one element
            else:
                col_penalties.append(-1)  # Indicate depleted column

        row_penalties = np.array(row_penalties)
        col_penalties = np.array(col_penalties)
        
        if np.max(row_penalties) >= np.max(col_penalties):
            i = np.argmax(row_penalties)
            if row_penalties[i] == -1:
                break  # Exit if row is depleted
            j = np.argmin(costs[i])
        else:
            j = np.argmax(col_penalties)
            if col_penalties[j] == -1:
                break  # Exit if column is depleted
            i = np.argmin(costs[:, j])
        
        allocation = min(supply[i], demand[j])
        solution[i, j] = allocation
        supply[i] -= allocation
        demand[j] -= allocation
        costs[i, j] = np.inf  # Set to inf to avoid re-allocation

    return solution

def print_solution(solution, method_name):
    print(f"\n{method_name} Solution:")
    print(pd.DataFrame(solution, columns=[f"D{j+1}" for j in range(solution.shape[1])],
                       index=[f"S{i+1}" for i in range(solution.shape[0])]))

def solve_transportation_problem(file_path):
    cost_matrix, supply, demand = read_transportation_data(file_path)
    print_solution(vogel_approximation_method(cost_matrix, supply, demand), "Vogel's Approximation Method")
solve_transportation_problem(file_path)



Vogel's Approximation Method Solution:
      D1    D2    D3    D4
S1  10.0   0.0  40.0   0.0
S2  10.0   5.0   0.0  45.0
S3   0.0  25.0   0.0   0.0
